In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_final=pd.read_csv('data/output/final_Data.csv')
df_final.info()

In [ ]:
df_final_data_copy = df_final.copy()
df_final_data_copy.drop(columns=['reuna','issuer','issue','check_digit','issuer+issue','gvkey'], inplace=True)
df_final_data_copy['datadate']= pd.to_datetime(df_final_data_copy['datadate'])
df_final_data_copy['fyear'] =  df_final_data_copy.apply(lambda row: row['fyear'] if not pd.isna(row['fyear']) else row['datadate'].year,axis=1)
df_final_data_copy['fyear'] = df_final_data_copy['fyear'].astype(int)
columns_of_ratios = ['liquidity','re/at','profitability','mkvalt/lt','AssetTurnoverRatio']

In [ ]:
# showing different counts of financial and industrial data
value_counts = df_final_data_copy['indfmt'].value_counts()
print(value_counts)
value_counts.plot(kind='bar')

In [ ]:
df_financial_data = df_final_data_copy[df_final_data_copy['indfmt']=='FS']
df_industrial_data = df_final_data_copy[df_final_data_copy['indfmt']=='INDL']

In [ ]:
nas_in_finanical_data = df_financial_data[columns_of_ratios].isna().sum()
print(nas_in_finanical_data)
nas_in_finanical_data.plot(kind='bar',title="NaNs Counts of financial")

In [ ]:
nas_in_industrial_data = df_industrial_data[columns_of_ratios].isna().sum()
print(nas_in_industrial_data)
nas_in_industrial_data.plot(kind='bar',title="NaNs Counts of industrial",xlabel='ratio',ylabel='Nans Counts')

In [ ]:
df_industrial_data['gsector'] = df_industrial_data['gsector'].replace([np.inf, -np.inf], np.nan)
df_industrial_data['gsector'] = df_industrial_data['gsector'].fillna(0)



In [ ]:
df_industrial_data['gsector'] = df_industrial_data['gsector'].astype(int)

In [ ]:
induatrial_value_countss = df_industrial_data['gsector'].value_counts()
print(induatrial_value_countss)
induatrial_value_countss.plot(kind='bar',title='Value Counts of industrial sectors',xlabel='sector code',ylabel='No of rows')

In [ ]:
# Group by 'gsector' and count the number of NaN values in the specified columns
nan_counts_by_ratio = df_industrial_data.groupby('gsector')[columns_of_ratios].apply(lambda x: x.isna().sum())
nan_counts = df_industrial_data.groupby('gsector')[columns_of_ratios].apply(lambda x: x.isna().sum().sum())
# Display the result
print(nan_counts_by_ratio)

nan_counts_by_ratio.plot(kind='bar')
plt.title('NaNs Counts By ratios per sector')
plt.xlabel('ratio')
plt.ylabel('Nans Counts')
plt.show()

print(nan_counts)
nan_counts.plot(kind='bar',title='NaNs Counts By Sector',xlabel='Sector',ylabel='Nans Counts')


In [ ]:
percentage_nan = (nan_counts / (induatrial_value_countss*5)) * 100
print(percentage_nan)
percentage_nan.plot(kind='bar',title='Percentage of NaN by sector',xlabel='sector code',ylabel='Percentage')



# Selecting Sector code 35 (Health Care) , as it has lowest percentage of NaN values, and also comparatively high number of rows ( 3rd position)

In [ ]:
df =df_industrial_data[df_industrial_data['gsector']==35]
df

In [ ]:
values = df['fyear'].value_counts()
print(values)
values.plot(kind='bar',title='Value Counts of years in sector 35',xlabel='year',ylabel='No of rows')


In [ ]:
df_new =df[(df['fyear']>2010)&(df['fyear']<2020)]


In [ ]:
df_new

In [ ]:
df_pivot1 = df_new.pivot_table(index=['cusip','conm','PERMNO','DLSTCD','HCOMNAM','HEXCD','BEGDAT','ENDDAT','ipodate'], columns='fyear', values=['liquidity','re/at','profitability','mkvalt/lt','AssetTurnoverRatio'])
df_pivot1.columns= [f'{year}-{ratio}' for ratio, year in df_pivot1.columns]
# Reset the index to make 'Company' a column again
df_pivot1.reset_index(inplace=True)

# Display the reshaped DataFrame
df_pivot1

In [ ]:
selected_columns = df_pivot1.iloc[:, 9:]
# Count the number of NaN values in each row
nan_counts_per_row = selected_columns.isna().sum(axis=1)
print("Number of NaN values in each row:")
print(nan_counts_per_row)

In [ ]:
df = pd.DataFrame(selected_columns)
print(df.shape)
print(df[(nan_counts_per_row>40)].shape[0])
print(df[(nan_counts_per_row>30)].shape[0])
print(df[(nan_counts_per_row>20)].shape[0])
print(df[(nan_counts_per_row>10)].shape[0])



In [ ]:
rows_to_remove = nan_counts_per_row[nan_counts_per_row > 30].index
df_pivot_after_dropping_nans= df_pivot1.drop(rows_to_remove)
df_pivot_after_dropping_nans

In [ ]:
selected_columns1 = df_pivot_after_dropping_nans.iloc[:, 9:]
# Count the number of NaN values in each row
nan_counts_per_row1 = selected_columns.isna().sum(axis=1)
print("Number of NaN values in each row:")
print(nan_counts_per_row1)
df = pd.DataFrame(selected_columns1)
print(df.shape)
print(df[(nan_counts_per_row1>40)].shape[0])
print(df[(nan_counts_per_row1>30)].shape[0])
print(df[(nan_counts_per_row1>20)].shape[0])
print(df[(nan_counts_per_row1>10)].shape[0])


## Imputing the entire data set df_pivot1

In [ ]:
import numpy as np
from py_soft_impute.soft_impute import SoftImpute
df_pivot_to_impute = df_pivot1.iloc[:, 9:]
df_pivot_to_impute.replace([np.inf, -np.inf], np.nan, inplace=True)
df_pivot_to_impute.fillna(0, inplace=True)

In [ ]:
clf = SoftImpute(J=2, lambda_=0.0)
fit = clf.fit(df_pivot_to_impute.values)
pivot_impute = clf.predict(df_pivot_to_impute.values)
pivot_impute

In [ ]:
new_df = pd.DataFrame(pivot_impute, columns=df_pivot_to_impute.columns)
df_pivot1_imputed= df_pivot1.copy()
df_pivot1_imputed.iloc[:, 9:] = new_df

In [ ]:
df_pivot1_imputed

In [ ]:
df_pivot1_imputed.to_csv('data/output/df_pivot1_imputed.csv', index=False)

## imputing the df after removing rows df_pivot_after_dropping_nans

In [ ]:
import numpy as np
from py_soft_impute.soft_impute import SoftImpute
df_pivot_after_dropping_nans_to_impute = df_pivot_after_dropping_nans.iloc[:, 9:]
df_pivot_after_dropping_nans_to_impute.replace([np.inf, -np.inf], np.nan, inplace=True)
df_pivot_after_dropping_nans_to_impute.fillna(0, inplace=True)

clf = SoftImpute(J=2, lambda_=0.0)
fit = clf.fit(df_pivot_after_dropping_nans_to_impute.values)
imputed_array = clf.predict(df_pivot_after_dropping_nans_to_impute.values)
imputed_array

In [ ]:
new_df1 = pd.DataFrame(imputed_array, columns=df_pivot_after_dropping_nans_to_impute.columns)
df_pivot_after_dropping_nans_imputed= df_pivot_after_dropping_nans.copy()
df_pivot_after_dropping_nans_imputed.iloc[:, 9:] = new_df1
df_pivot_after_dropping_nans_imputed

In [ ]:
df_pivot_after_dropping_nans_imputed.to_csv('data/output/df_pivot_after_dropping_nans_imputed.csv', index=False)